In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS fleetops_project.silver;

CREATE OR REPLACE TEMP VIEW vw_drivers_latest AS
SELECT * EXCEPT(rn)
FROM (
    SELECT 
        driver_id, name, cnh, cnh_type, contract_date, updated_at,
        ROW_NUMBER() OVER (PARTITION BY driver_id ORDER BY _ingested_at DESC) as rn
    FROM fleetops_project.bronze.drivers
) WHERE rn = 1;

CREATE TABLE IF NOT EXISTS fleetops_project.silver.drivers
USING DELTA
AS SELECT * FROM vw_drivers_latest WHERE 1=0;

MERGE WITH SCHEMA EVOLUTION INTO fleetops_project.silver.drivers t
USING vw_drivers_latest s
ON t.driver_id = s.driver_id
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *;